In [1]:
# Let's get started with our challenge
# Firstly, we ought to create a new directory made specially for this occasion
from pathlib import Path
import shutil
desafio = Path.cwd() / 'Desafio'
desafio.mkdir()

In [4]:
# Now, let's move this file into the new directory
file =  Path.cwd() / 'desafio_email_1.ipynb'
shutil.move(str(file), desafio)

'/Users/felipeveiga/Documents/Jupyter USP/Python Lira/pathlib e shutil/Desafio/desafio_email_1.ipynb'

In [7]:
# The shopping mall's files have been sent to my email address, so I'll have to move them into my current directory
from imap_tools import MailBox, AND
import os
from zipfile import ZipFile

# To do this I will have to log in my email address
username = os.environ.get('EMAIL_USER')
password = os.environ.get('MOT_USER')

# Let's log in
gmail = MailBox('imap.gmail.com').login(username, password)

# I will face the following issue: Iterating over my whole mail box would take a lot of time
# So, aspiring efficiency, I will have to create an algorithm that filters our search
# A first parameter I could give to the code is the date the messages we are looking for were sent
# This will be sufficient to shrink my search range
import datetime
hoje = datetime.datetime.now().date()
emails_procurados = gmail.fetch(AND(date = hoje))

for email in emails_procurados:
    # The emails I am looking for must have an attachment
    if len(email.attachments) != 0:
        
        


0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
2
0
0


KeyboardInterrupt: 

In [3]:
for file in os.listdir(current):
    print(file)

202001_Morumbi_SP.csv
202003_Shopping Leblon_RJ.csv
201801_Buriti Shopping_GO.csv
201801_Goiania Shopping_GO.csv
202001_Plaza Shopping_RJ.csv
201804_Shopping Cidade_MG.csv
201803_Shopping Tijuca_RJ.csv
202004_Rio Sul_RJ.csv
201902_Shopping Cidade_MG.csv
201802_Cidade de Sao Paulo_SP.csv
201904_JK Iguatemi_SP.csv
202004_Shopping Leblon_RJ.csv
202001_JK Iguatemi_SP.csv
201902_Independˆncia Shopping_MG.csv
201902_Ibirapuera_SP.csv
201801_JK Iguatemi_SP.csv
201803_Shopping Cidade_MG.csv
201804_Shopping Tijuca_RJ.csv
202003_Rio Sul_RJ.csv
201803_BH Shopping_MG.csv
202001_Minas Shopping_MG.csv
201802_Ibirapuera_SP.csv
202002_Bourbon_SP.csv
201902_Shopping Tijuca_RJ.csv
202001_Barra Shopping_RJ.csv
202003_BH Shopping_MG.csv
202003_Bourbon_SP.csv
201804_Plaza Shopping_RJ.csv
202002_Rio Sul_RJ.csv
.DS_Store
201903_Buriti Shopping_GO.csv
202002_JK Iguatemi_SP.csv
202002_Iguatemi_SP.csv
201801_Independˆncia Shopping_MG.csv
202001_Cidade de Sao Paulo_SP.csv
202003_Plaza Shopping_RJ.csv
201802_JK I

In [2]:
from zipfile import ZipFile
import os
from pathlib import Path
import shutil

a = '/Users/felipeveiga/Documents/Jupyter USP/Projetos/Email 1'
os.chdir(a)

#arquivos = Path.cwd() / 'excel_desafio_email1'
current = Path.cwd() / 'excel_desafio_email1'

with ZipFile('Email 1/Desafio_email1.zip', 'w') as zip:
    for file in os.listdir(current):
        print(file)
    #for file in arquivos.iterdir():
        zip.write(file) 
        

202001_Morumbi_SP.csv
202003_Shopping Leblon_RJ.csv
201801_Buriti Shopping_GO.csv
201801_Goiania Shopping_GO.csv
202001_Plaza Shopping_RJ.csv
201804_Shopping Cidade_MG.csv
201803_Shopping Tijuca_RJ.csv
202004_Rio Sul_RJ.csv
201902_Shopping Cidade_MG.csv
201802_Cidade de Sao Paulo_SP.csv
201904_JK Iguatemi_SP.csv
202004_Shopping Leblon_RJ.csv
202001_JK Iguatemi_SP.csv
201902_Independˆncia Shopping_MG.csv
201902_Ibirapuera_SP.csv
201801_JK Iguatemi_SP.csv
201803_Shopping Cidade_MG.csv
201804_Shopping Tijuca_RJ.csv
202003_Rio Sul_RJ.csv
201803_BH Shopping_MG.csv
202001_Minas Shopping_MG.csv
201802_Ibirapuera_SP.csv
202002_Bourbon_SP.csv
201902_Shopping Tijuca_RJ.csv
202001_Barra Shopping_RJ.csv
202003_BH Shopping_MG.csv
202003_Bourbon_SP.csv
201804_Plaza Shopping_RJ.csv
202002_Rio Sul_RJ.csv
.DS_Store
201903_Buriti Shopping_GO.csv
202002_JK Iguatemi_SP.csv
202002_Iguatemi_SP.csv
201801_Independˆncia Shopping_MG.csv
202001_Cidade de Sao Paulo_SP.csv
202003_Plaza Shopping_RJ.csv
201802_JK I

KeyboardInterrupt: 

In [1]:
from pathlib import Path
import shutil
import os
#a = Path.cwd() / 'excel_desafio_email1'
a = '/Users/felipeveiga/Documents/Jupyter USP/Projetos/Email 1/excel_desafio_email1'
os.chdir(a)
print(a)

/Users/felipeveiga/Documents/Jupyter USP/Projetos/Email 1/excel_desafio_email1


In [ ]:
from zipfile import ZipFile
import os
from pathlib import Path
import shutil


arquivos = Path.cwd() / 'excel_desafio_email1'
#current = Path.cwd()

with ZipFile('arquivos_shopping.zip', 'w') as zip:
    for file in arquivos.iterdir():
        print(file)
    #for file in arquivos.iterdir():
        zip.write(file) 
        

In [ ]:
import smtplib
from email.message import EmailMessage
import mimetypes
current = Path.cwd()

me = os.environ.get('EMAIL_USER')
mot = os.environ.get('MOT_USER')
for file in current.iterdir():
    if file.suffix == '.zip':
        msg = EmailMessage()
        msg['From'] = me
        msg['To'] = me
        msg['Subject'] = 'Shopping Malls Documents'
        msg.set_content(' Dear Felipe\n Here you can find the xlsx files demanded.\n Have a nice day,\n Felipe')
        with open(file, 'rb') as excel:
            file_content = excel.read()
            maintype, subtype = mimetypes.guess_types(file)[0].split('/')
            msg.add_attachment(file_content, maintype=maintype, subtype=subtype, filename = file.name)
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(me, mot)
            smtp.send_message(msg)
        


In [8]:
print(' Dear Felipe\n Here you can find the xlsx files demanded.\n Have a nice day,\n Felipe')

 Dear Felipe
 Here you can find the xlsx files demanded.
 Have a nice day,
 Felipe
